# HAN模型简单代码复现

*参考资料：*
* [源代码链接](https://github.com/taishan1994/pytorch_HAN)

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from model import *
from utils import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = "cpu"

In [3]:
adj_list, fea_list, y_train, y_val, y_test, train_mask, val_mask, test_mask, my_data = load_data_dblp()

2125 300 600
y_train:(3025, 3), y_val:(3025, 3), y_test:(3025, 3), train_idx:(1, 2125), val_idx:(1, 300), test_idx:(1, 600)


In [4]:
# rownetworks: ['PAP','PLP']
for i in adj_list:
    print(i.shape)

(3025, 3025)
(3025, 3025)


In [5]:
print('节点数目,特征维度：',fea_list[0].shape)
print(y_train.shape)
print(y_train[3000])
print(train_mask.shape)

节点数目,特征维度： (3025, 1870)
(3025, 3)
[0. 0. 1.]
(3025,)


In [6]:
nb_nodes = fea_list[0].shape[0] #节点数目 3025
ft_size = fea_list[0].shape[1] #特征的维度 1870
nb_classes = y_train.shape[1]  #标签的维度 3
print(nb_nodes,ft_size,nb_classes)

3025 1870 3


In [7]:
fea_list = [torch.transpose(torch.from_numpy(fea[np.newaxis]),2,1).to(device) for fea in fea_list]

In [8]:
print(fea_list[0].shape)


torch.Size([1, 1870, 3025])


In [9]:
adj_list = [adj[np.newaxis] for adj in adj_list]
y_train = y_train[np.newaxis]
y_val = y_val[np.newaxis]
y_test = y_test[np.newaxis]

In [10]:
print(adj_list[0].shape)
print(y_train.shape)


(1, 3025, 3025)
(1, 3025, 3)


In [11]:
my_labels = my_data['my_labels']
train_my_labels = my_data['train_my_labels']
val_my_labels = my_data['val_my_labels']
test_my_labels = my_data['test_my_labels']

In [12]:
print(my_labels.shape)
print(train_my_labels.shape)
print(val_my_labels.shape)
print(test_my_labels.shape)


(3025,)
(2125,)
(300,)
(600,)


In [13]:
biases_list = [torch.transpose(torch.from_numpy(adj_to_bias(adj, [nb_nodes], nhood=1)),2,1).to(device) for adj in adj_list]
print(len(biases_list))

2


In [14]:
biases_list[0]

tensor([[[-0.0000e+00, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
          -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -0.0000e+00, -1.0000e+09,  ..., -1.0000e+09,
          -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -0.0000e+00,  ..., -1.0000e+09,
          -1.0000e+09, -1.0000e+09],
         ...,
         [-1.0000e+09, -1.0000e+09, -1.0000e+09,  ..., -0.0000e+00,
          -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
          -0.0000e+00, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
          -1.0000e+09, -0.0000e+00]]], dtype=torch.float64)

In [15]:
biases_list[0].shape

torch.Size([1, 3025, 3025])

In [16]:
h1 = np.array([[0.0,1.0,0.0,0,1],[1.0,0.0,1.0,1,0],[0.0,1.0,0.0,0,0],[0,1,0,0,1],[1,0,0,1,0]])
h1 = h1[np.newaxis]
print(h1.shape)
print(h1)
adj_to_bias(h1, [5], nhood=2)

(1, 5, 5)
[[[0. 1. 0. 0. 1.]
  [1. 0. 1. 1. 0.]
  [0. 1. 0. 0. 0.]
  [0. 1. 0. 0. 1.]
  [1. 0. 0. 1. 0.]]]


array([[[-0.e+00, -0.e+00, -0.e+00, -0.e+00, -0.e+00],
        [-0.e+00, -0.e+00, -0.e+00, -0.e+00, -0.e+00],
        [-0.e+00, -0.e+00, -0.e+00, -0.e+00, -1.e+09],
        [-0.e+00, -0.e+00, -0.e+00, -0.e+00, -0.e+00],
        [-0.e+00, -0.e+00, -1.e+09, -0.e+00, -0.e+00]]])

In [17]:
dataset = 'acm'
featype = 'fea'
checkpt_file = 'pre_trained/{}/{}_allMP_multi_{}_.ckpt'.format(dataset, dataset, featype)
print('model: {}'.format(checkpt_file))

model: pre_trained/acm/acm_allMP_multi_fea_.ckpt


### 模型参数设置

In [18]:
# training params
batch_size = 1 # 批大小
nb_epochs = 200 # 迭代次数
patience = 100  # 
lr = 0.005  # learning rate学习率
l2_coef = 0.0005  # weight decay

# numbers of hidden units per each attention head in each layer
hid_units = [8] # 隐藏单元
n_heads = [8, 1]  # additional entry for the output layer 注意力数
residual = False

In [19]:
inputs = torch.randn(1,1870,3025)
ret = Attn_head(1870, 8, biases_list[0], activation=nn.ELU())
result = ret(inputs)
print(result.shape)

torch.Size([1, 8, 3025])


In [20]:
print("fea_list[0].shape:",fea_list[0].shape)
print("biases_list[0].shape:",biases_list[0].shape)
print(len(fea_list))
print(len(biases_list))

fea_list[0].shape: torch.Size([1, 1870, 3025])
biases_list[0].shape: torch.Size([1, 3025, 3025])
3
2


In [21]:
for inputs,biases in zip(fea_list,biases_list):
    print(inputs.shape, biases.shape)

torch.Size([1, 1870, 3025]) torch.Size([1, 3025, 3025])
torch.Size([1, 1870, 3025]) torch.Size([1, 3025, 3025])


In [22]:
model = HeteGAT_multi(inputs_list=fea_list,nb_classes=nb_classes,nb_nodes=nb_nodes,attn_drop=0.5,
                      ffd_drop=0.0,bias_mat_list=biases_list,hid_units=hid_units,n_heads=n_heads,
                      activation=nn.ELU(),residual=False)

model.to(device)

HeteGAT_multi(
  (activation): ELU(alpha=1.0)
  (layers): Sequential(
    (0): Attn_head(
      (conv1): Conv1d(1870, 8, kernel_size=(1,), stride=(1,), bias=False)
      (conv2_1): Conv1d(8, 1, kernel_size=(1,), stride=(1,), bias=False)
      (conv2_2): Conv1d(8, 1, kernel_size=(1,), stride=(1,), bias=False)
      (leakyrelu): LeakyReLU(negative_slope=0.01)
      (softmax): Softmax(dim=1)
      (in_dropout): Dropout(p=0.0, inplace=False)
      (coef_dropout): Dropout(p=0.5, inplace=False)
      (activation): ELU(alpha=1.0)
    )
    (1): Attn_head(
      (conv1): Conv1d(1870, 8, kernel_size=(1,), stride=(1,), bias=False)
      (conv2_1): Conv1d(8, 1, kernel_size=(1,), stride=(1,), bias=False)
      (conv2_2): Conv1d(8, 1, kernel_size=(1,), stride=(1,), bias=False)
      (leakyrelu): LeakyReLU(negative_slope=0.01)
      (softmax): Softmax(dim=1)
      (in_dropout): Dropout(p=0.0, inplace=False)
      (coef_dropout): Dropout(p=0.5, inplace=False)
      (activation): ELU(alpha=1.0)
    )


In [23]:
result211 = model.forward(fea_list)
result211.shape

torch.Size([3025, 3])

In [24]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(params=model.parameters(),lr=lr,betas=(0.9,0.99),weight_decay=0.0)

train_my_labels = torch.from_numpy(train_my_labels).long().to(device)
val_my_labels = torch.from_numpy(val_my_labels).long().to(device)
test_my_labels = torch.from_numpy(test_my_labels).long().to(device)

train_mask = np.where(train_mask == 1)[0]
val_mask = np.where(val_mask == 1)[0]
test_mask = np.where(test_mask == 1)[0]
train_mask = torch.from_numpy(train_mask).to(device)
val_mask = torch.from_numpy(val_mask).to(device)
test_mask = torch.from_numpy(test_mask).to(device)

In [25]:
def train():
    model.train()
    correct = 0
    outputs = model(fea_list)
    train_mask_outputs = torch.index_select(outputs,0,train_mask)
    _, preds = torch.max(train_mask_outputs.data,1)
    #print(preds)
    #print(train_my_labels)
    loss = criterion(train_mask_outputs,train_my_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    correct += torch.sum(preds == train_my_labels).to(torch.float32)
    acc = correct / len(train_my_labels)
    #val_loss,val_acc = test("val",val_mask,val_my_labels,epoch)
    #test_acc = test("test",test_mask,test_my_labels,epoch)
    #test_acc_history.append(test_acc)
    #print("epoch:{:03d}, loss:{:.4f}, TrainAcc:{:.4F}, ValLoss:{:.4f}, ValAcc:{:.4f}".format(epoch,loss,acc,val_loss,val_acc))
    return loss,acc

In [26]:
def test(mode,mask,label):
    model.eval()
    with torch.no_grad():
        correct = 0.0
        outputs = model(fea_list)
        mask_outputs = torch.index_select(outputs,0,mask)
        _, preds = torch.max(mask_outputs,1)
        loss = criterion(mask_outputs,label)
        correct += torch.sum(preds == label).to(torch.float32)
        if mode == "val":
            acc = correct / len(label)
        elif mode == "test":
            acc = correct / len(label)
        else:
            print("请输入合法的mode: val/test")
            return
        #print("[{}]>>>>>  [epoch]:{:03d}, [loss]:{:.4f}, [acc]:{:.4F}".format(mode,epoch,loss,acc))
    return loss,acc

In [27]:
def main():
    train_loss_history = []
    train_acc_history = []
    val_loss_history = []
    val_acc_history = []
    print("训练节点个数：",len(train_my_labels))
    print("验证节点个数：",len(val_my_labels))
    print("测试节点个数：",len(test_my_labels))
    for epoch in range(1,20):
        train_loss,train_acc = train()
        val_loss,val_acc = test("val",val_mask,val_my_labels)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc)
        print("epoch:{:03d}, loss:{:.4f}, TrainAcc:{:.4F}, ValLoss:{:.4f}, ValAcc:{:.4f}".format(epoch,train_loss,train_acc,val_loss,val_acc))
    test_loss,test_acc = test("test",test_mask,test_my_labels)
    print("TestAcc:{:.4f}".format(test_acc))
    
    return train_loss_history, train_acc_history, val_loss_history, val_acc_history

In [28]:
train_loss_history, train_acc_history, val_loss_history, val_acc_history = main()

训练节点个数： 2125
验证节点个数： 300
测试节点个数： 600


epoch:001, loss:1.0963, TrainAcc:0.3586, ValLoss:1.1904, ValAcc:0.2167
epoch:002, loss:1.1018, TrainAcc:0.3642, ValLoss:1.1412, ValAcc:0.2367
epoch:003, loss:1.0792, TrainAcc:0.4475, ValLoss:1.1102, ValAcc:0.3367
epoch:004, loss:1.0702, TrainAcc:0.4080, ValLoss:1.0841, ValAcc:0.3700
epoch:005, loss:1.0505, TrainAcc:0.5327, ValLoss:1.0961, ValAcc:0.3567
epoch:006, loss:1.0220, TrainAcc:0.6136, ValLoss:1.1178, ValAcc:0.3167
epoch:007, loss:1.0022, TrainAcc:0.6160, ValLoss:1.1279, ValAcc:0.3133
epoch:008, loss:0.9787, TrainAcc:0.6452, ValLoss:1.1242, ValAcc:0.3200
epoch:009, loss:0.9567, TrainAcc:0.6725, ValLoss:1.1078, ValAcc:0.3500
epoch:010, loss:0.9305, TrainAcc:0.7096, ValLoss:1.0825, ValAcc:0.4133
epoch:011, loss:0.9039, TrainAcc:0.7384, ValLoss:1.0550, ValAcc:0.4633
epoch:012, loss:0.8676, TrainAcc:0.7760, ValLoss:1.0294, ValAcc:0.5267
epoch:013, loss:0.8364, TrainAcc:0.7962, ValLoss:1.0070, ValAcc:0.5500
epoch:014, loss:0.7989, TrainAcc:0.8113, ValLoss:0.9844, ValAcc:0.5667
epoch: